In [10]:
import pandas as pd
import numpy as np
from nltk import TreebankWordTokenizer
import os
import string
from keras.preprocessing.sequence import pad_sequences

from keras import layers
from keras.layers import merge
from keras import Input
from keras.models import Model, model_from_json
from keras.optimizers import Adadelta
from keras.callbacks import EarlyStopping
import keras.backend as K
import matplotlib.pyplot as plt
import re

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize

Using TensorFlow backend.


In [11]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords

STOP_WORDS = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer() 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-03-28 13:33:38--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-03-28 13:33:38--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-03-28 13:33:39--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [3]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
def load_glove():
  embedding = {}
  f = open('/content/glove.6B.50d.txt',encoding='utf-8')
  for line in f:
    values = line.split()
    word = values[0]
    coeff = np.asarray(values[1:],dtype = 'float32')
    embedding[word] = coeff
  f.close()
  return embedding

In [0]:
def get_vector(word):
  if word in embeddings:
    return embeddings[word]

In [0]:
def vectorise_sentence(sentence):
    tokens = tokenizer.tokenize(sentence)
    vector = []
    for token in tokens:
        if token not in string.punctuation:
            token_vector = get_vector(token)
            if token_vector is not None:
                vector.append(token_vector)
                
    return vector

In [0]:
def vectorize_df(df):
  vectors_a = [vectorise_sentence(s) for s in df.sentence_A]
  vectors_b = [vectorise_sentence(s) for s in df.sentence_B]
  scores = ((df.relatedness_score-1)/4).tolist()
  return vectors_a,vectors_b,scores

In [0]:
def test_custom(sentence1,sentence2,max_len,model):
  vectors_a = [vectorise_sentence(sentence1)]
  vectors_a = pad_sequences(vectors_a,padding ='post',maxlen=max_len)

  vectors_b = [vectorise_sentence(sentence2)]
  vectors_b = pad_sequences(vectors_b,padding = 'post',maxlen=max_len)

  normalized_score = model.predict([vectors_a,vectors_b])
  return normalized_score*4 + 1

In [0]:
def lemmatize_word(text): 
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in text] 
    return lemmas 

In [0]:
def clean_text(lines):
  '''Clean text by removing unnecessary characters and altering the format of words.'''
  re_print = re.compile('[^%s]' % re.escape(string.printable))
  cleaned = list()
  for text in lines:
      text = text.lower()
      
      text = re.sub(r"i'm", "i am", text)
      text = re.sub(r"he's", "he is", text)
      text = re.sub(r"she's", "she is", text)
      text = re.sub(r"it's", "it is", text)
      text = re.sub(r"that's", "that is", text)
      text = re.sub(r"what's", "that is", text)
      text = re.sub(r"where's", "where is", text)
      text = re.sub(r"how's", "how is", text)
      text = re.sub(r"\'ll", " will", text)
      text = re.sub(r"\'ve", " have", text)
      text = re.sub(r"\'re", " are", text)
      text = re.sub(r"\'d", " would", text)
      text = re.sub(r"\'re", " are", text)
      text = re.sub(r"won't", "will not", text)
      text = re.sub(r"can't", "cannot", text)
      text = re.sub(r"n't", " not", text)
      text = re.sub(r"n'", "ng", text)
      text = re.sub(r"'bout", "about", text)
      text = re.sub(r"'til", "until", text)
      text = re.sub(r"[$-()\"#/@;:<>{}`+=~|.!?,'*-]", "", text)
                        
      text = text.split()
      text = [re_print.sub('', w) for w in text]
      text = [token for token in text if token not in STOP_WORDS]
      text = lemmatize_word(text)
      cleaned.append(' '.join(text))
                        
  return cleaned

In [0]:
embeddings = load_glove()

In [19]:

!pip install wget
import wget
url = 'https://raw.githubusercontent.com/jaydeepthik/Siamese-Sentence-Similarity/master/SICK/SICK.txt'
wget.download(url,'SICK.txt')

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=142698fb384fb2365248f1de0843e96863b1aeec83ae2f5576d9cd59c180d785
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


'SICK.txt'

In [0]:
tokenizer = TreebankWordTokenizer()

In [0]:
filepath='/content/SICK.txt'


In [0]:
df = pd.read_csv(filepath, sep='\t')[['sentence_A', 'sentence_B', 'relatedness_score', 'SemEval_set']]

In [23]:

df.head()

,sentence_A,sentence_B,relatedness_score,SemEval_set
0,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,TRAIN
1,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,TRAIN
2,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,TRAIN
3,The young boys are playing outdoors and the ma...,There is no boy playing outdoors and there is ...,3.6,TRIAL
4,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,TRAIN


In [0]:
train_data = df[df['SemEval_set'] == 'TRAIN']
dev_data = df[df['SemEval_set']=='TRIAL']
test_data = df[df['SemEval_set']=='TEST']



In [25]:
train_data.head()

,sentence_A,sentence_B,relatedness_score,SemEval_set
0,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,TRAIN
1,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,TRAIN
2,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,TRAIN
4,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,TRAIN
8,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,TRAIN


In [0]:
df['sentence_A'] = clean_text(df['sentence_A'])
df['sentence_B'] = clean_text(df['sentence_B'])

In [0]:
sentence_A = train_data['sentence_A'].tolist()



In [0]:
sentence_B = train_data['sentence_B'].tolist()

In [0]:
def get_scores(df):
  scores = ((df.relatedness_score-1)/4).tolist()
  return scores

In [0]:
scores = get_scores(train_data)

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentence_A + sentence_B)


In [0]:
se_a = tokenizer.texts_to_sequences(sentence_A)
se_b = tokenizer.texts_to_sequences(sentence_B)


In [0]:
max_a = max(len(s) for s in se_a)
max_b = max(len(s) for s in se_b)
max_len = max([max_a,max_b])

In [0]:
train_a = pad_sequences(se_a,maxlen=max_len,padding='post',truncating='post')
train_b = pad_sequences(se_b,maxlen=max_len, padding = 'post',truncating='post')

In [0]:
import numpy as np
def create_train_dev_set(sentence_A,sentence_B, max_sequence_length,scores,validation_split_ratio):
    """
    Create training and validation dataset
    Args:
        tokenizer (keras.preprocessing.text.Tokenizer): keras tokenizer object
        sentences_pair (list): list of tuple of sentences pairs
        is_similar (list): list containing labels if respective sentences in sentence1 and sentence2
                           are same or not (1 if same else 0)
        max_sequence_length (int): max sequence length of sentences to apply padding
        validation_split_ratio (float): contain ratio to split training data into validation data
    Returns:
        train_data_1 (list): list of input features for training set from sentences1
        train_data_2 (list): list of input features for training set from sentences2
        labels_train (np.array): array containing similarity score for training data
        leaks_train(np.array): array of training leaks features
        val_data_1 (list): list of input features for validation set from sentences1
        val_data_2 (list): list of input features for validation set from sentences1
        labels_val (np.array): array containing similarity score for validation data
        leaks_val (np.array): array of validation leaks features
    """
    #sentences1 = [x[0].lower() for x in sentences_pair]
    #sentences2 = [x[1].lower() for x in sentences_pair]
    #train_sequences_1 = tokenizer.texts_to_sequences(sentences1)
    #train_sequences_2 = tokenizer.texts_to_sequences(sentences2)
    leaks = [[len(set(x1)), len(set(x2)), len(set(x1).intersection(x2))]
             for x1, x2 in zip(se_a,se_b)]

    #train_padded_data_1 = pad_sequences(train_sequences_1, maxlen=max_sequence_length)
    #train_padded_data_2 = pad_sequences(train_sequences_2, maxlen=max_sequence_length)
    train_labels = np.array(scores)
    leaks = np.array(leaks)

    shuffle_indices = np.random.permutation(np.arange(len(train_labels)))
    train_data_1_shuffled = train_a[shuffle_indices]
    train_data_2_shuffled = train_b[shuffle_indices]
    train_labels_shuffled = train_labels[shuffle_indices]
    leaks_shuffled = leaks[shuffle_indices]

    dev_idx = max(1, int(len(train_labels_shuffled) * validation_split_ratio))

    #del train_padded_data_1
    #del train_padded_data_2
    #gc.collect()

    train_data_1, val_data_1 = train_data_1_shuffled[:-dev_idx], train_data_1_shuffled[-dev_idx:]
    train_data_2, val_data_2 = train_data_2_shuffled[:-dev_idx], train_data_2_shuffled[-dev_idx:]
    labels_train, labels_val = train_labels_shuffled[:-dev_idx], train_labels_shuffled[-dev_idx:]
    leaks_train, leaks_val = leaks_shuffled[:-dev_idx], leaks_shuffled[-dev_idx:]

    return train_data_1, train_data_2, labels_train, leaks_train, val_data_1, val_data_2, labels_val, leaks_val




In [0]:
train_data_x1, train_data_x2, train_labels, leaks_train,val_data_x1, val_data_x2, val_labels, leaks_val = create_train_dev_set(sentence_A,sentence_B, max_len,scores,validation_split_ratio =0.2)

In [0]:
vocab_size=len(tokenizer.word_index) + 1

In [0]:
num_words_input=len(tokenizer.word_index)+1
word2idx_input=tokenizer.word_index

num_words=num_words_input
embedding_matrix=np.zeros((num_words,50))
for word,i in word2idx_input.items():
    if i<num_words_input:
        embedding_vector=embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i]=embedding_vector

In [0]:
import tensorflow as tf
embed_layer = tf.keras.layers.Embedding(vocab_size,50,weights = [embedding_matrix],input_length = max_len)

In [0]:
LSTM = tf.keras.layers.LSTM(50, unit_forget_bias=True, kernel_initializer='he_normal', kernel_regularizer='l2', name='lstm_layer')
left_input = tf.keras.layers.Input(shape=(max_len,))



In [0]:
left_input_1 = embed_layer(left_input)

In [0]:
left_output = LSTM(left_input_1)

In [0]:
right_input = tf.keras.layers.Input(shape=(max_len,))
right_input_1 = embed_layer(right_input)
right_output = LSTM(right_input_1)

In [0]:
import tensorflow.keras.backend as K
def exponent_neg_manhattan_distance(x, hidden_size=50):
    ''' Helper function for the similarity estimate of the LSTMs outputs '''
    return K.exp(-K.sum(K.abs(x[:,:hidden_size] - x[:,hidden_size:]), axis=1, keepdims=True))

def exponent_neg_cosine_distance(x, hidden_size=50):
    ''' Helper function for the similarity estimate of the LSTMs outputs '''
    leftNorm = K.l2_normalize(x[:,:hidden_size], axis=-1)
    rightNorm = K.l2_normalize(x[:,hidden_size:], axis=-1)
    return K.exp(K.sum(K.prod([leftNorm, rightNorm], axis=0), axis=1, keepdims=True))

In [0]:
concats = tf.keras.layers.concatenate([left_output,right_output],axis=-1)

In [0]:
main_output = tf.keras.layers.Lambda(exponent_neg_manhattan_distance, output_shape=(1,))(concats)


In [0]:
model = tf.keras.models.Model([left_input, right_input],concats)

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mean_squared_error', metrics=['mae','mse'])

In [124]:
 model.fit([train_data_x1, train_data_x2, leaks_train], train_labels,
                  validation_data=([val_data_x1, val_data_x2, leaks_val], val_labels),
                  epochs=50, batch_size=32)

Epoch 1/50
 85/111 [=====================>........] - ETA: 1s - loss: 0.0642 - mae: 0.1973 - mse: 0.0642

KeyboardInterrupt: ignored

In [0]:
model = model.save('model.h5')


In [0]:
sentence_1 = ['What can make Physics easy to learn?']
sentence_2=['How can you make physics easy to learn?']

In [0]:
sentence_1=clean_text(sentence_1)

In [0]:
sentence_2=clean_text(sentence_2)

In [0]:
sentence_1=tokenizer.texts_to_sequences(sentence_1)
sentence_2 = tokenizer.texts_to_sequences(sentence_2)


In [0]:
sentence1 = pad_sequences(sentence_1,padding='post',maxlen = max_len)
sentence2 = pad_sequences(sentence_2,padding='post',maxlen = max_len)


In [0]:
def create_test_data(tokenizer, sentence1,sentence2, max_sequence_length):
    """
    Create training and validation dataset
    Args:
        tokenizer (keras.preprocessing.text.Tokenizer): keras tokenizer object
        test_sentences_pair (list): list of tuple of sentences pairs
        max_sequence_length (int): max sequence length of sentences to apply padding
    Returns:
        test_data_1 (list): list of input features for training set from sentences1
        test_data_2 (list): list of input features for training set from sentences2
    """
    #test_sentences1 = [x[0].lower() for x in test_sentences_pair]
    #test_sentences2 = [x[1].lower() for x in test_sentences_pair]

    #test_sequences_1 = tokenizer.texts_to_sequences(test_sentences1)
    #test_sequences_2 = tokenizer.texts_to_sequences(test_sentences2)
    leaks_test = [[len(set(x1)), len(set(x2)), len(set(x1).intersection(x2))]
                  for x1, x2 in zip(sentence_1,sentence_2)]

    leaks_test = np.array(leaks_test)
    #test_data_1 = pad_sequences(test_sequences_1, maxlen=max_sequence_length)
    #test_data_2 = pad_sequences(test_sequences_2, maxlen=max_sequence_length)

    return sentence1,sentence2, leaks_test

In [0]:
sentence1,sentence2, leaks_test = create_test_data(tokenizer, sentence1,sentence2, max_len)

In [0]:
import tensorflow as tf
from tensorflow.keras.models import load_model

In [107]:
model = load_model('model.h5')


In [108]:
preds = list(model.predict([sentence1,sentence2, leaks_test], verbose=1).ravel())

1/1 [==============================] - 0s 1ms/step


In [0]:
test_sentence_pairs = [('What can make Physics easy to learn?','How can you make physics easy to learn?')]

In [0]:
results = [(x, y, z) for (x, y), z in zip(test_sentence_pairs, preds)]

In [112]:
from operator import itemgetter
results.sort(key=itemgetter(2), reverse=True)
print(results)


[('What can make Physics easy to learn?', 'How can you make physics easy to learn?', 0.6220899)]
